In [ ]:
%load_ext autoreload
%autoreload 2


import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

import pandas as pd

from analysis_variables import *

import os

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'
# At Siena on villa-straylight
#topdir = '/home/bellis/'


# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


In [ ]:
spmodes = ['-999', '998', '1005', '1235', '1237']

plt.figure(figsize=(16,8))
for i,spmode in enumerate(spmodes):

    print("-----------------------")
    
    mask = data['spmode']==spmode
    fl = data[mask]['Lambda0FlightLen']

    maskfl = fl>=0
    #x = ak.num(data[mask]['Lambda0_unc_Mass'])
    x = ak.num(data[mask]['BMass'])

    xflcut = ak.num(data[mask]['BMass'][maskfl])

    plt.subplot(2,3,i+1)
    h = plt.hist(x,bins=11,range=(-0.5,10.5), label=spmode)
    hflcut = plt.hist(xflcut,bins=11,range=(-0.5,10.5), label='FL cut', alpha=0.3)

    plt.yscale('log')
    plt.legend()
    
    vals = h[0]
    valsflcut = hflcut[0]
    for i in range(len(vals)):
        print(i,vals[i],valsflcut[i])
    
    print()
    n2 = vals[2]
    nother = sum(vals[0:2]) + sum(vals[3:]) 
    print(f"{spmode:6s} {n2}  {nother}   {n2+nother} {100*n2/(n2+nother):.2f}% {100*nother/(n2+nother):.2f}%")
    print()
    n1 = valsflcut[1]
    nother = sum(valsflcut[0:1]) + sum(valsflcut[2:]) 
    print(f"{spmode:6s} {n1}  {nother}  {n1+nother}  {100*n1/(n1+nother):.2f}% {100*nother/(n1+nother):.2f}%")
    print()


# Function to study duplicates

In [ ]:
# Check nBs versus nLams
#plt.figure(figsize=(16,12))
for i,spmode in enumerate(spmodes):

    print("-----------------------")
    
    #spmode = '1005'
    mask = data['spmode']==spmode
    x = ak.num(data[mask]['Lambda0_unc_Mass'])
    y = ak.num(data[mask]['BMass'])

    nTrue = len(x[x==y])
    nFalse = len(x[x!=y])

    print(f"{spmode:6s} {nTrue:8d} {nFalse:8d}")

    '''
    plt.subplot(3,2,i+1)
    h = plt.hist(x,bins=11,range=(-0.5,10.5), label=spmode)
    plt.yscale('log')
    plt.legend()
    
    vals = h[0]
    for i in range(len(vals)):
        print(i,vals[i])
    
    print()
    n2 = vals[2]
    nother = sum(vals[0:2]) + sum(vals[3:]) 
    print(f"{spmode:6s} {n2}  {nother}    {100*nother/n2:.2f}%")
    '''
#plt.legend()

In [ ]:
def spherical_to_cartesian(p3_spher):
    r = p3_spher[0]
    costh = p3_spher[1]
    phi = p3_spher[2]
    
    #print(p3_spher)
    #print(costh)
    theta = np.arccos(costh)

    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)

    pmag = np.sqrt(x**2 + y**2 + z**2)
    print(pmag, r)

    return x,y,z, pmag

def mass_from_spherical(p4s_spherical):

    etot,px,py,pz = 0,0,0,0
    
    for p4s_sph in p4s_spherical:
        #print("Here")
        #print(p4s_sph, p4s_sph[1:])
        x,y,z,pmag = spherical_to_cartesian(p4s_sph[1:])
        mass = p4s_sph[0]
        e = np.sqrt(mass**2 + pmag**2)
        
        etot += e
        px += x
        py += y
        pz += z

    #print("p4: ", etot, px, py, pz)
    m2 = etot**2 - (px**2 + py**2 + pz**2)
    if m2>=0:
        return np.sqrt(m2)
    else:
        return -np.sqrt(-m2)
    



In [ ]:
output = ""
for f in data.fields:
    if len(output)<80:
        output += f"{f:16s} "
    else:
        print(output)
        output = f"{f:16s} "

In [ ]:
def duplicate_issue_summary(data, spmode, number_of_Bs=2):
    
    mask = data['spmode']==spmode
    
    lamd1idx = data[mask]['Lambda0d1Idx']
    lamd2idx = data[mask]['Lambda0d2Idx']

    lamd1lund = data[mask]['Lambda0d1Lund']
    lamd2lund = data[mask]['Lambda0d2Lund']

    lamp3 = data[mask]['Lambda0p3']
    lamcosth = data[mask]['Lambda0costh']
    lamphi = data[mask]['Lambda0phi']
    lammass = data[mask]['Lambda0_unc_Mass']

    lamvx = data[mask]['Lambda0Vtxx']
    lamvy = data[mask]['Lambda0Vtxy']
    lamvz = data[mask]['Lambda0Vtxz']

    lamfl = data[mask]['Lambda0FlightLen']

    Bvx = data[mask]['B_con_Vtxx']
    Bvy = data[mask]['B_con_Vtxy']
    Bvz = data[mask]['B_con_Vtxz']
    
    pp3 = data[mask]['pp3']
    pcosth = data[mask]['pcosth']
    pphi = data[mask]['pphi']
    ptrkidx = data[mask]['pTrkIdx']

    pip3 = data[mask]['pip3']
    picosth = data[mask]['picosth']
    piphi = data[mask]['piphi']
    pitrkidx = data[mask]['piTrkIdx']

    trkp3 = data[mask]['TRKp3']
    trkcosth = data[mask]['TRKcosth']
    trkphi = data[mask]['TRKphi']

    
    BMass = data[mask]['BMass']

    
    Bd1idx = data[mask]['Bd1Idx']
    Bd2idx = data[mask]['Bd2Idx']

    Bd1lund = data[mask]['Bd1Lund']
    Bd2lund = data[mask]['Bd2Lund']

    nBs = ak.num(data[mask]['BMass'])

    #print(nBs)
    
    # Find the first instance when the number of Bs is what was passed in
    indices = ak.where(nBs==number_of_Bs)
    idx = indices[0][0]

    print(f"{spmode}")
    for n in range(number_of_Bs):
        print(f"B index {n}    {BMass[idx][n]:.10f}")
        bd1idx = Bd1idx[idx][n]
        bd2idx = Bd2idx[idx][n]
        print(f"B dau#1: {bd1idx}  {Bd1lund[idx][n]}    B dau#2: {bd2idx}  {Bd2lund[idx][n]} ")
        
        print(f"d1 lambda p3/costh/phi: {lamp3[idx][bd1idx]:.10f}  {lamcosth[idx][bd1idx]:.10f}  {lamphi[idx][bd1idx]:.10f}")
        print(f"d2 proton p3/costh/phi: {pp3[idx][bd2idx]:.10f}  {pcosth[idx][bd2idx]:.10f}  {pphi[idx][bd2idx]:.10f}")
        trkidx = ptrkidx[idx][bd2idx]
        print(f"d2 TRK    p3/costh/phi: {trkp3[idx][trkidx]:.10f}  {trkcosth[idx][trkidx]:.10f}  {trkphi[idx][trkidx]:.10f}")

        print()
        print(f"B          vx/vy/vz:     {Bvx[idx][bd1idx]:.10f}  {Bvy[idx][bd1idx]:.10f}  {Bvz[idx][bd1idx]:.10f}")
        print(f"d1 Lambda0 vx/vy/vz:     {lamvx[idx][bd1idx]:.10f}  {lamvy[idx][bd1idx]:.10f}  {lamvz[idx][bd1idx]:.10f}")
        print()

        p41 = [1.11568, lamp3[idx][bd1idx], lamcosth[idx][bd1idx], lamphi[idx][bd1idx]]
        p42 = [0.938272,    pp3[idx][bd2idx], pcosth[idx][bd2idx], pphi[idx][bd2idx]]
        #print("THERE:", p41)
        m = mass_from_spherical([p41, p42])
        print("B my calc: ", m)
        print()
        
        # Lambda daughters        
        ld1idx = lamd1idx[idx][bd1idx]
        ld2idx = lamd2idx[idx][bd1idx]
        ld1lund = lamd1lund[idx][bd1idx]
        ld2lund = lamd2lund[idx][bd1idx]
        print(f"Lambda0 dau#1: {ld1idx}  {ld1lund}    Lambda0 dau#2: {ld2idx}  {ld2lund}     {lammass[idx][bd1idx]:.10f}  {lamfl[idx][bd1idx]:.6f}")


        print(f"d1 proton p3/costh/phi: {pp3[idx][ld1idx]:.10f}  {pcosth[idx][ld1idx]:.10f}  {pphi[idx][ld1idx]:.10f}")
        print(f"d2 pion   p3/costh/phi: {pip3[idx][ld2idx]:.10f}  {picosth[idx][ld2idx]:.10f}  {piphi[idx][ld2idx]:.10f}")

        trkidx = ptrkidx[idx][ld1idx]
        print(f"d1 TRK    p3/costh/phi: {trkp3[idx][trkidx]:.10f}  {trkcosth[idx][trkidx]:.10f}  {trkphi[idx][trkidx]:.10f}")
        trkidx = pitrkidx[idx][ld2idx]
        print(f"d2 TRK    p3/costh/phi: {trkp3[idx][trkidx]:.10f}  {trkcosth[idx][trkidx]:.10f}  {trkphi[idx][trkidx]:.10f}")

        
        #print(ld1idx, ld2idx)

        p41 = [0.938272, pp3[idx][ld1idx], pcosth[idx][ld1idx], pphi[idx][ld1idx]]
        p42 = [0.139570,    pip3[idx][ld2idx], picosth[idx][ld2idx], piphi[idx][ld2idx]]
        #print("THERE:", p41)
        m = mass_from_spherical([p41, p42])
        print("Lambda0 mycalc: ", m)
        

        
        print("\n\n")
    
    return nBs

#nBs = duplicate_issue_summary(data, '1235', 2)
nBs = duplicate_issue_summary(data, '1005', 2)



In [ ]:
nBs.to_numpy().tolist().index(3)

In [ ]:
ak.where(nBs==3)[0][0]

In [ ]:
nlam = data['nLambda0']
lamfl = data['Lambda0FlightLen']
lammass= data['Lambda0_unc_Mass']
spmode= data['spmode']


d1idx = data['Lambda0d1Idx']
d2idx = data['Lambda0d2Idx']

d1lund = data['Lambda0d1Lund']
d2lund = data['Lambda0d2Lund']

Bd1idx = data['Bd1Idx']
Bd2idx = data['Bd2Idx']

Bd1lund = data['Bd1Lund']
Bd2lund = data['Bd2Lund']



plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(nlam);

plt.subplot(1,2,2)
plt.hist(nlam[nlam!=2]);

In [ ]:
print(lamfl)
print(lammass)
print(spmode)

print()

print(d1idx)
print(d1lund)
print()

print(d2idx)
print(d2lund)
print()

print(Bd1idx)
print(Bd1lund)
print()

print(Bd2idx)
print(Bd2lund)
print()

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.flatten(lamfl),bins=100);

plt.subplot(1,2,2)
plt.hist(ak.flatten(lamfl[mask]), bins=100);

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.num(lamfl));

plt.subplot(1,2,2)
plt.hist(ak.num(lamfl[mask]));

In [ ]:
mask = nlam == 2

x1 = d1idx[mask][:,0]
x2 = d1idx[mask][:,1]

x3 = d2idx[mask][:,0]
x4 = d2idx[mask][:,1]

x5 = Bd2idx[mask][:,0]
x6 = Bd2idx[mask][:,1]

x7 = lammass[mask][:,0]
x8 = lammass[mask][:,1]


print(x1)
print(x2)
print()

print("Lambda0 d1")
print(len(x1))
print(len(x1[x1 != x2]))
print(x1[x1 != x2])
print(x2[x1 != x2])
print(spmode[x1 != x2])
print()

print("Lambda0 d2")
print(len(x3))
print(len(x3[x3 != x4]))
print(x3[x3 != x4])
print(x4[x3 != x4])
print(spmode[x3 != x4])
print()

print("B d2 (proton)")
print(len(x5))
print(len(x5[x5 != x6]))
print(x5[x5 != x6])
print(x6[x5 != x6])
print(spmode[x5 != x6])
print()

print("Lambda0 mass")
print(len(x7))
print(len(x7[x7 != x8]))
print(x7[x7 != x8])
print(x8[x7 != x8])


print(x7[x7 != x8][4])
print(x8[x7 != x8][4])

print(x7[x7 != x8][5])
print(x8[x7 != x8][5])

print(x7[x7 != x8][7])
print(x8[x7 != x8][7])


print(spmode[x7 != x8])
print()


In [ ]:
#data.fields

In [ ]:
mask = nlam == 2

x1 = d1idx[mask][:,0]
x2 = d1idx[mask][:,1]

x3 = d2idx[mask][:,0]
x4 = d2idx[mask][:,1]

x5 = Bd2idx[mask][:,0]
x6 = Bd2idx[mask][:,1]

x7 = lammass[mask][:,0]
x8 = lammass[mask][:,1]


n = 25

ppmag, pcosth, pphi = data['pp3'], data['pcosth'], data['pphi']
pipmag, picosth, piphi = data['pip3'], data['picosth'], data['piphi']

print(x1[n], x2[n], x6[n] , ppmag[n], pipmag[n], pcosth[n], picosth[n], pphi[n], piphi[n])
#ppmag

In [ ]:
mask = nlam == 2

print(d1idx[mask])

x1 = d1idx[mask][:,1]

x1